In [ ]:
import numpy as np
import tensorflow as tf
from functools import wraps
from tensorflow import keras
from sklearn.datasets import load_iris
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from tensorflow.keras.datasets import mnist

## Funktionen generieren und Accuracy auswerten

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Verarbeiten der Daten
x_train = x_train.reshape(60000, 784).astype("float64") / 255
x_test = x_test.reshape(10000, 784).astype("float64") / 255

y_train = y_train.astype("float64")
y_test = y_test.astype("float64")

# Reservieren 10.000 Samples zur Validierung
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

In [ ]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='log.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args):
        #logging.info('Ran with args: {}'.format(args))
        result = orig_func(*args)
        logging.info(result)
        return result
    
        #logging.info(orig_func(*args))
        #return orig_func(*args)

    return wrapper

def my_timer(orig_func):
    import time
    import logging
    logging.basicConfig(filename='log.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args):
        t1 = time.time()
        result = orig_func(*args)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        logging.info('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

In [ ]:
print(fit(x_train, y_train))

print(predict(x_test))

# Unit Test

In [ ]:
class Normalize(object): 
    def normalize(self, x_train, x_test):
        self.scaler = MinMaxScaler()
        x_train = self.scaler.fit_transform(x_train)
        x_test  = self.scaler.transform(x_test)
        return (x_train, x_test) 
    
    def inverse(self, x_train, x_val, x_test):
        x_train = self.scaler.inverse_transform(x_train)
        x_test  = self.scaler.inverse_transform(x_test)
        return (x_train, x_test)   

In [ ]:
class TheAlgorithm(object):
  
    @my_logger
    @my_timer
    def __init__(self, x_train, y_train, x_test, y_test):  
      self.x_train, self.y_train, self.x_test, self.y_test = x_train, y_train, x_test, y_test    
        
    @my_logger
    @my_timer
    def fit(self): 
        normalizer = Normalize()
        self.x_train, self.x_test = normalizer.normalize(self.x_train, self.x_test)   
        train_samples = self.x_train.shape[0]
        self.classifier = LogisticRegression(
            C=50. / train_samples,
            multi_class='multinomial',
            penalty='l1',
            solver='saga',
            tol=0.1,
            class_weight='balanced',
            )
        self.classifier.fit(self.x_train, self.y_train)
        self.train_y_predicted = self.classifier.predict(self.x_train)
        self.train_accuracy = np.mean(self.train_y_predicted.ravel() == self.y_train.ravel()) * 100
        self.train_confusion_matrix = confusion_matrix(self.y_train, self.train_y_predicted)        
        return self.train_accuracy
    
    @my_logger
    @my_timer
    def predict(self):
        self.test_y_predicted = self.classifier.predict(self.x_test) 
        self.test_accuracy = np.mean(self.test_y_predicted.ravel() == self.y_test.ravel()) * 100 
        self.test_confusion_matrix = confusion_matrix(self.y_test, self.test_y_predicted)        
        self.report = classification_report(self.y_test, self.test_y_predicted)
        print("Classification report for classifier:\n %s\n" % (self.report))
        return self.test_accuracy

In [ ]:
import unittest
import time

class TestInput(unittest.TestCase):
  
    @classmethod
    def setUpClass(cls):  
        pass

    @classmethod
    def tearDownClass(cls): 
        pass

    def setUp(self):
        print('setUp') 
        #X, y = download()
        #splitRatio = 60000
        #self.X_train, self.y_train, self.X_test, self.y_test = split(X,y,splitRatio) 
        
        self.train_accuracy = 72.442  # 73.44 fit()
        
        self.test_accuracy = 73.57000000000001  # 74 predict()
        
        self.train_runtime = 19
        
    def tearDown(self):
        # print('tearDown')
        pass
        
    def test_fit(self):     
        np.random.seed(31337)
        self.ta = TheAlgorithm(x_train, y_train, x_test, y_test)
        
        # Zeit messen
        start_fit= time.time()
        result = self.ta.fit()
        duration_fit = time.time() - start_fit
        
        #self.assertEqual(self.ta.fit(), self.train_accuracy) 
        self.assertEqual(result, self.train_accuracy) 
        
        #Zeit verifizieren < 120 %
        self.assertLessEqual(duration_fit, 1.2 * 18)
  
    def test_predict(self):
        np.random.seed(31337)
        self.ta = TheAlgorithm(x_train, y_train, x_test, y_test)
        self.ta.fit()
        self.assertEqual(self.ta.predict(), self.test_accuracy)
      
if __name__ == '__main__':
  
    #run tests 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)